# **作業說明**
# (這是Udacity關於A/B Test的期末專題)

Udacity希望了解，在免費14天試學網頁上，除了要信用卡資訊外，還想了解學生願意花多少小時學。如果少於某門檻(5小時)，就建議學生不要註冊，免費聽聽影音就好，免得浪費資源，降低學習成功率。

我們的題目是，增加這個頁面，是否對Gross Conversion(GC)和Net Conversion (NC)在統計學上(Alpha=0.05，Power=0.8)有幫助(d=0.01/0.0075)，亦即統計上的顯著(Significant)。

CI = click 數目

GC = 註冊數/CI (聽了建議仍然註冊的比例)

NC = 繳費數/CI (14天之後繳費且繼續的比例)

我們期待GC比原來下降，但NC不降，這表示省去資源但收入不降。

檔名：ab-tests-with-python.ipynb

**作業目標**

1.   經由範例程式，學習A/B Test 的步驟
2.   最低樣本數的計算方法
3.   自行開發信賴區間計算函數













In [32]:
alpha=0.05
beta=0.8

In [1]:
#載入程式庫
import math as mt
import numpy as np
import pandas as pd
from scipy.stats import norm

In [2]:
#將基礎數據放入字典

baseline = {"Cookies":40000,
            "Clicks":3200,            #click 數目
            "Enrollments":660,
            "CTP":0.08,
            "GConversion":0.20625,    #Enrollment/Click (聽了建議仍然註冊的比例) 
            "Retention":0.53,
            "NConversion":0.109313}   #繳費數/CI (14 天之後繳費且繼續的比例)

In [8]:
#調整大小到以Cookie為基準

baseline["Cookies"] = 5000
baseline["Clicks"]=baseline["Clicks"]*(5000/40000)
baseline["Enrollments"]=baseline["Enrollments"]*(5000/40000)
baseline

{'Cookies': 5000,
 'Clicks': 50.0,
 'Enrollments': 10.3125,
 'CTP': 0.08,
 'GConversion': 0.20625,
 'Retention': 0.53,
 'NConversion': 0.109313}

In [9]:
# Gross Conversion (GC)


GC={}

GC["d_min"]=0.01

GC["p"]=baseline["GConversion"]   #p is given in this case - or we could calculate it from enrollments/clicks
GC["n"]=baseline["Clicks"]

GC["sd"]=round(mt.sqrt((GC["p"]*(1-GC["p"]))/GC["n"]),4)
GC["sd"]

0.0572

In [10]:
# Retention(R) 

R={}

R["d_min"]=0.01

R["p"]=baseline["Retention"]
R["n"]=baseline["Enrollments"]

R["sd"]=round(mt.sqrt((R["p"]*(1-R["p"]))/R["n"]),4)
R["sd"]

0.1554

In [12]:
# Net Conversion (NC)

NC={}

NC["d_min"]=0.0075

NC["p"]=baseline["NConversion"]
NC["n"]=baseline["Clicks"]

NC["sd"]=round(mt.sqrt((NC["p"]*(1-NC["p"]))/NC["n"]),4)
NC["sd"]

0.0441

In [13]:
#計算 Z-score
def get_z_score(alpha):
    return norm.ppf(alpha)

# 得到兩個(A/B)標準差
def get_sds(p,d):
    sd1=mt.sqrt(2*p*(1-p))
    sd2=mt.sqrt(p*(1-p)+(p+d)*(1-(p+d)))
    sds=[sd1,sd2]
    return sds

# 求Sample Size
def get_sampSize(sds,alpha,beta,d):
    n=pow((get_z_score(1-alpha/2)*sds[0]+get_z_score(1-beta)*sds[1]),2)/pow(d,2)
    return n

In [14]:
GC["d"]=0.01
R["d"]=0.01
NC["d"]=0.0075

In [18]:
# Let's get an integer value for simplicity
GC["SampSize"]=round(get_sampSize(get_sds(GC["p"],GC["d"]),0.05,0.2,GC["d"]))
GC["SampSize"]  #25835.0

GC["SampSize"]=round(GC["SampSize"]/0.08*2)
GC["SampSize"]

645875.0

In [19]:
# Getting a nice integer value
R["SampSize"]=round(get_sampSize(get_sds(R["p"],R["d"]),0.05,0.2,R["d"]))
R["SampSize"]  #39087.0

R["SampSize"]=R["SampSize"]/0.08/0.20625*2
R["SampSize"]

4737818.181818182

In [20]:
# Getting a nice integer value
NC["SampSize"]=round(get_sampSize(get_sds(NC["p"],NC["d"]),0.05,0.2,NC["d"]))
NC["SampSize"]

NC["SampSize"]=NC["SampSize"]/0.08*2
NC["SampSize"]

685325.0

In [24]:
# 載入數據
control=pd.read_csv("./control_data.csv")
experiment=pd.read_csv("./experiment_data.csv")
display(control.head())
display(experiment.head())

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


In [25]:
display(control.info())
display(experiment.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 5 columns):
Date           37 non-null object
Pageviews      37 non-null int64
Clicks         37 non-null int64
Enrollments    23 non-null float64
Payments       23 non-null float64
dtypes: float64(2), int64(2), object(1)
memory usage: 1.6+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 5 columns):
Date           37 non-null object
Pageviews      37 non-null int64
Clicks         37 non-null int64
Enrollments    23 non-null float64
Payments       23 non-null float64
dtypes: float64(2), int64(2), object(1)
memory usage: 1.6+ KB


None

In [23]:
pageviews_cont=control['Pageviews'].sum()

pageviews_exp=experiment['Pageviews'].sum()

pageviews_total=pageviews_cont+pageviews_exp

print ("number of pageviews in control:", pageviews_cont)
print ("number of Pageviewsin experiment:" ,pageviews_exp)

number of pageviews in control: 345543
number of Pageviewsin experiment: 344660


In [28]:
# Count the total clicks from complete records only
# Enrollment中有些遺失值

clicks_cont=control["Clicks"].loc[control["Enrollments"].notnull()].sum()
clicks_exp=experiment["Clicks"].loc[experiment["Enrollments"].notnull()].sum()

print ("number of clicks in control:", clicks_cont)
print ("number of clicks experiment:" ,clicks_exp)

number of clicks in control: 17293
number of clicks experiment: 17260


In [33]:
#Gross Conversion - number of enrollments divided by number of clicks

enrollments_cont=control["Enrollments"].sum()
enrollments_exp=experiment["Enrollments"].sum()

GC_cont=enrollments_cont/clicks_cont
GC_exp=enrollments_exp/clicks_exp

GC_pooled=(enrollments_cont+enrollments_exp)/(clicks_cont+clicks_exp)
GC_sd_pooled=mt.sqrt(GC_pooled*(1-GC_pooled)*(1/clicks_cont+1/clicks_exp))

GC_ME=round(get_z_score(1-alpha/2)*GC_sd_pooled,4)
GC_diff=round(GC_exp-GC_cont,4)

print("The change due to the experiment is",GC_diff*100,"%")
print("Confidence Interval: [",GC_diff-GC_ME,",",GC_diff+GC_ME,"]")
print ("The change is statistically significant if the CI doesn't include 0. In that case, it is practically significant if",-GC["d_min"],"is not in the CI as well.")

The change due to the experiment is -2.06 %
Confidence Interval: [ -0.0292 , -0.012 ]
The change is statistically significant if the CI doesn't include 0. In that case, it is practically significant if -0.01 is not in the CI as well.


In [34]:
#Net Conversion - number of payments divided by number of clicks

payments_cont=control["Payments"].sum()
payments_exp=experiment["Payments"].sum()

NC_cont=payments_cont/clicks_cont
NC_exp=payments_exp/clicks_exp

NC_pooled=(payments_cont+payments_exp)/(clicks_cont+clicks_exp)
NC_sd_pooled=mt.sqrt(NC_pooled*(1-NC_pooled)*(1/clicks_cont+1/clicks_exp))

NC_ME=round(get_z_score(1-alpha/2)*NC_sd_pooled,4)
NC_diff=round(NC_exp-NC_cont,4)

print("The change due to the experiment is",NC_diff*100,"%")
print("Confidence Interval: [",NC_diff-NC_ME,",",NC_diff+NC_ME,"]")
print ("The change is statistically significant if the CI doesn't include 0. In that case, it is practically significant if",NC["d_min"],"is not in the CI as well.")

The change due to the experiment is -0.49 %
Confidence Interval: [ -0.0116 , 0.0018000000000000004 ]
The change is statistically significant if the CI doesn't include 0. In that case, it is practically significant if 0.0075 is not in the CI as well.


# **作業**
# 經由範例程式碼，熟悉A/B Test的步驟

請同學逐步跟隨程式了解A/B Test步驟

# **作業 嘗試以函數算出樣本數**

In [35]:
import statsmodels.stats.api as sms
from math import ceil 

#### 假設檢定:
$H_0: P_{cont}=P_{exp}$

$H_1: P_{cont}-P_{exp}=d_{min}$

In [37]:
effect_size = sms.proportion_effectsize(GC["p"]-1.0*GC["d_min"], GC["p"]+0.0*GC["d_min"])
required_n = sms.NormalIndPower().solve_power(
    effect_size, 
    power=0.8, 
    alpha=0.05, 
    ratio=1
    ) 

required_n = ceil(required_n) 
print (effect_size,'\n',required_n)  

-0.02494345647889673 
 25231


# **作業** 自行開發雙樣本比例的信賴區間函數


In [38]:
import scipy.stats as stats

![jupyter](./notion.png)

![jupyter](./cofi_interval.png)

In [52]:
#兩獨立母體比例差之區間估計

def two_proprotions_confint(success_a, size_a, success_b, size_b, significance = 0.05):

    # P1、P2
    prop_a = success_a / size_a
    prop_b = success_b / size_b
    print('P1:', round(prop_a,4),'\n', 'P2:', round(prop_b,4))
    
    # 標準誤
    var = prop_a * (1 - prop_a) / size_a + prop_b * (1 - prop_b) / size_b
    se = np.sqrt(var)
    print('Standard Error:', round(se,4))
    
    # z critical value
    confidence = 1 - significance  #信心水準、這裡為95%
    z = stats.norm(loc = 0, scale = 1).ppf(confidence + significance / 2)
    print('z-score:', z)

    # standard formula for the confidence interval
    # point-estimtate +- z * standard-error
    prop_diff = prop_b - prop_a
    print('P1-P2:', round(prop_diff,4))
    
    confint = prop_diff + np.array([-1, 1]) * z * se
    return prop_diff, confint

In [53]:
two_proprotions_confint(enrollments_cont, clicks_cont, enrollments_exp, clicks_exp, significance = 0.05)

P1: 0.2189 
 P2: 0.1983
Standard Error: 0.0044
z-score: 1.959963984540054
P1-P2: -0.0206


(-0.020554874580361565, array([-0.02912016, -0.01198959]))